In [0]:
%run ../utils/_DeltaLakeInit

In [0]:
%sql
CREATE OR REPLACE TABLE LakeDb.SourceUsers
AS
SELECT Id, DisplayName, LastAccessDate, Location, Reputation FROM Silver.DeltaUsers

In [0]:
%sql
DROP TABLE IF EXISTS LakeDb.TargetUsers;
--CREATE OR REPLACE TABLE LakeDb.SourceUsers SHALLOW CLONE Silver.DeltaUsers;


In [0]:
%sql
SELECT * FROM LakeDb.SourceUsers

# Scenario: Target table has one more column

In [0]:
%sql
CREATE OR REPLACE TABLE LakeDb.TargetUsers
(
    Id int,
    DisplayName STRING,
    LastAccessDate TIMESTAMP,
    Reputation INT,
    NewColumn INT
) USING DELTA

In [0]:
%sql
SELECT * FROM LakeDb.TargetUsers

In [0]:
%sql
DESCRIBE DETAIL LakeDb.TargetUsers

In [0]:
TargetUsersPath = 'dbfs:/user/hive/warehouse/lakedb.db/targetusers'

In [0]:
%sql
SELECT Id, DisplayName, LastAccessDate, Reputation FROM LakeDb.SourceUsers LIMIT 3

In [0]:
_sqldf.write.format("delta").mode("append").save(TargetUsersPath)

In [0]:
%sql
SELECT * FROM LakeDb.TargetUsers;

# Scenario: Source table has an extra column
(which doesn't match target table)

In [0]:
%sql
SELECT Id, DisplayName, LastAccessDate, Reputation, `Location` FROM LakeDb.SourceUsers 
WHERE `Location` = 'Stevenage'
LIMIT 3

In [0]:
# Attempt to append new DataFrame (with new column) to existing table
_sqldf.write.format("delta").mode("append").save(TargetUsersPath)

In [0]:
# Add the mergeSchema option
_sqldf.write.format("delta") \
           .option("mergeSchema", "true") \
           .mode("append") \
           .save(TargetUsersPath)

In [0]:
%sql
SELECT * FROM LakeDb.TargetUsers

# Explicitly update schema (SQL)

In [0]:
%sql
ALTER TABLE LakeDb.TargetUsers ADD COLUMNS (Age INT COMMENT 'Age of user')

In [0]:
%sql
ALTER TABLE LakeDb.TargetUsers ADD COLUMNS (WebSite STRING AFTER Reputation)

## RENAME COLUMN
This feature is available in `Delta Lake 1.2.0` and above. This feature is currently **experimental**.

In [0]:
%sql
-- RENAME COLUMN
ALTER TABLE LakeDb.TargetUsers RENAME COLUMN Age TO Aged

https://www.databricks.com/blog/2019/09/24/diving-into-delta-lake-schema-enforcement-evolution.html  
https://docs.delta.io/latest/delta-batch.html#schema-validation  
https://learn.microsoft.com/en-gb/azure/databricks/delta/update-schema#merge-schema-evolution